# Implementacja algorytmu genetycznego

# Modele - Implementacja </br>

- Barbara Pobiedzińska 400781
- Tomasz Brania 400642
- Marcin Biela 302959 </br>

Grupa 1a

In [1]:
import numpy as np
import pandas as pd
import random
import copy
from typing import List


TODO  struktury danych:

xn - stan “lodówki” na początku x0 = 0

yn - decyzja czy w danym dniu idziemy na zakupy

n - n-ty dzień

an - ograniczenie pojemności plecaka w danym dniu

b - prędkość zużywania zapasów

ci - tabela wagi wartościowości

N - ilość dni do ustalenia terminarza


In [2]:
#dane
n_records = 10

tabela_produktow = [f"Produkt{i+1}" for i in range(n_records)]
print(tabela_produktow)

# tabele zczytywac z excela c(wartociowosc od wagi od czegos dodatkowego np kalorie)
# ograniczenie plecaka tez (czy robic funkcje ktora wylicza ktore dni sa niehandlowe (niedziele i swieta))

['Produkt1', 'Produkt2', 'Produkt3', 'Produkt4', 'Produkt5', 'Produkt6', 'Produkt7', 'Produkt8', 'Produkt9', 'Produkt10']


In [3]:
# tabela_wartosciowosc_kalorie = np.random.randint(low=400, high=1000, size=n_records)

tabela_wartosciowosc_kalorie = np.round(np.random.normal(1000, 300, size=n_records))
tabela_wartosciowosc_przydatnosc = np.random.randint(low=0, high=2, size=n_records)
tabela_wag = np.round(np.random.random(n_records) * 0.7, 2)

print(tabela_wartosciowosc_kalorie)
print(tabela_wartosciowosc_przydatnosc)
print(tabela_wag)

[1505.  783. 1179.  659. 1197. 1296.  982. 1326.  740. 1040.]
[1 0 1 1 0 0 1 0 1 0]
[0.32 0.14 0.61 0.06 0.07 0.23 0.52 0.1  0.37 0.35]


# Dokumentacja danych odnoscie produktów

Dane o produktach mamy przedstawione w DataFramie w postaci:</br>
 - [waga, kaloryczność, przydatność]

In [4]:
# tabela_merge = {}
# for i in range(len(tabela_produktow)):
#   tabela_merge[tabela_produktow[i]] = [tabela_wag[i], tabela_wartosciowosc_kalorie[i], tabela_wartosciowosc_przydatnosc[i]]

# # print(tabela_merge)

# df = pd.DataFrame(tabela_merge).transpose()
# df.rename(columns={0: "waga", 1: "kaloryczność", 2:"przydatnosc"})

tabela_merge = {}
for i in range(len(tabela_produktow)):
  tabela_merge[tabela_produktow[i]] = [tabela_wag[i], tabela_wartosciowosc_kalorie[i]]

# print(tabela_merge)

df = pd.DataFrame(tabela_merge).transpose()
df.rename(columns={0: "waga", 1: "kaloryczność"})


,waga,kaloryczność
Produkt1,0.32,1505.0
Produkt2,0.14,783.0
Produkt3,0.61,1179.0
Produkt4,0.06,659.0
Produkt5,0.07,1197.0
Produkt6,0.23,1296.0
Produkt7,0.52,982.0
Produkt8,0.10,1326.0
Produkt9,0.37,740.0
Produkt10,0.35,1040.0


In [5]:
df.to_numpy()

array([[3.200e-01, 1.505e+03],
       [1.400e-01, 7.830e+02],
       [6.100e-01, 1.179e+03],
       [6.000e-02, 6.590e+02],
       [7.000e-02, 1.197e+03],
       [2.300e-01, 1.296e+03],
       [5.200e-01, 9.820e+02],
       [1.000e-01, 1.326e+03],
       [3.700e-01, 7.400e+02],
       [3.500e-01, 1.040e+03]])

In [6]:
## Klasa opisująca stan lodówki w czasie
# Posiada następujące metody: </br>
# - __init__, która przyjmuje jako parametry terminarz oraz liste produktow
# - generate_initial_solution - generuje rozwiązanie początkowe uwzględniając 
# ograniczenia dotyczące maksymalnej dopuszczalnej pojemnosci lodowki i plecaka
# - check_kalorie - Jeżeli bilans kaloryczny po wybraniu poczatkowych produktow
# jest mniejsz od 0 to dodaje n najlżejszych produktów o sumarycznej
# kaloryczności przekraczającej niedobór kaloryczny w danym dniu. 

In [7]:
# Ograniczenia
class lodowka():
  poczatkowy_stan_lodowki = 0 # poczatkowy stan lodowki

  maksymalna_poj_lodowki = 20 # maksymalna ilosc produktow w lodowce
  maks_liczba = 1 # maksymalna ilosc tego samego produktu
  N = 365 #tbd
  max_poj_plecaka = 7 # kg maksymalna pojemnosc_plecaka
  zapotrz_kal = 3000 # Zapotrzebowanie kaloryczne w danym dniu - w każdym dniu tyle samo
  kryterium_stopu = 100 # maksymalna ilosc iteracji

  def __init__(self, terminarz, lista_produktow):
    """
    parameters:
    terminarz - Przechowuje informacje w postaci (data, dzien_tygodnia, maksymalna_dopuszczalna_waga)
    jeżeli maksymalna_dopuszczalna_waga = 0 wtedy w danym dniu nie idziemy do sklepu w przeciwnym wypadku jest równe max_poj_plecaka

    lista produktow - przechowuje informacja w postaci np.ndarray, gdzie pierwsze

    initial_solution przyjmuje to co zwraca metoda generate_initial_solution()
    """
    self.terminarz = terminarz
    self.lista_produktow = lista_produktow
    self.initial_solution = self.generete_initial_solution()
    # self.aktualny_stan_lodowki = x0
    # self.aktualne_zapotrzebowanie_kaloryczne = zapotrz_kal
    self.tabu_list = [] # Lista tabu
    self.best_solution = self.initial_solution

  def generete_initial_solution(self) -> np.ndarray:
    """
    Returns:
    initial_solution (list): Rozwiązanie początkowe, baza do kolejnych kroków.
    Kolejne elementy initial_solution oznaczają kolejne dni terminarza.
    Przykładowe elementy listy:
    [1, [0, 0, 1, 0, 0, 1, 0, 1, 1, 1], 0]
    [0, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -19.0]
    gdzie:  initial_solution[i][0]: decyzja o pójściu na zakupy
            initial_solution[i][1]: lista wziętych produktów
            initial_solution[i][2]: bilans kalorii

    """
    initial_solution = []  # Rozwiązanie początkowe
    self.lista_produktow = np.array(self.lista_produktow)
    aktualny_stan_lodowki = self.poczatkowy_stan_lodowki
    zawartosc_lodowki = []
    for i in range(len(self.terminarz)):
      aktualny_stan_plecaka = 0
      if self.terminarz[i][2] != 0:
        teoretyczna_lista_zakupow = random.sample(range(10), 10)
        lista_1 = [0]*len(self.lista_produktow)
        count = 0
        while True and count < len(self.lista_produktow):
            indeks_produktu_ktory_bierzemy = teoretyczna_lista_zakupow[count]
            waga_produktu = self.lista_produktow[indeks_produktu_ktory_bierzemy][0]

            #sprawdzenie czy produkt który zamierzamy wziąć spełnia ograniczenia lodówki i plecaka:
            if aktualny_stan_lodowki + 1 <= self.maksymalna_poj_lodowki and aktualny_stan_plecaka + waga_produktu <= self.max_poj_plecaka:
              lista_1[indeks_produktu_ktory_bierzemy] = 1
              aktualny_stan_lodowki += 1
              aktualny_stan_plecaka += waga_produktu
              count += 1
            else:
              break
        initial_solution.append([1, lista_1 , 0])
        zawartosc_lodowki.append(lista_1)
      else:
        initial_solution.append([ 0, [0]*len(self.lista_produktow) , 0])
      
      wiersz_copy = copy.deepcopy(initial_solution[-1])
      #aktualizacja zużycia produktow (sprawdzenie w których dniach będzie niedobór kaloryczny (poprawiane następnie w check_kalorie())):
      aktualne_zuzycie = 0
      while aktualne_zuzycie < self.zapotrz_kal:
        if (len(np.nonzero(zawartosc_lodowki)[0])) > 0:
          id_row = np.nonzero(zawartosc_lodowki)[0][0]
          id_col = np.nonzero(zawartosc_lodowki)[1][0]
          jedzony_produkt = self.lista_produktow[id_col]
          zawartosc_lodowki[id_row][id_col] = 0
          kalorycznosc = jedzony_produkt[1]
          aktualne_zuzycie += kalorycznosc
          aktualny_stan_lodowki -= 1
        else:
          wiersz_copy[2] += aktualne_zuzycie - self.zapotrz_kal #aktualizacja bilansu kalorii (zaznaczenie niedoboru)
          break

      initial_solution[-1] = wiersz_copy
    return initial_solution


  def check_kalorie(self):
    """
    Metoda sprawdza czy produkty, które w danym dniu wybraliśmy spełniają nasze
    ograniczenie dotyczące zapotrzebowania dziennego na kalorie.

    Jeśli niedobor w danym dniu czyli (suma kaloryczności poszczególnych
    produktów) - (zapotrzebowanie dzienne) <  0 to wtedy wybieramy produkty o
    najmniejszej możliwej wadze dopóki niedobór będzie większy od 0.
    note: w tym momencie nie sprawdzamy czy po dadaniu produktu przekroczyliśmy
    maksymalna dopuszczalna pojemnosc plecaka max_poj_plecaka
    """
    for i in range(len(self.initial_solution)):
      if self.initial_solution[i][2] < 0: # jeśli niedobor jest mniejszy od 0
        self.initial_solution[i][0] = 1 #trzeba bedzie pojsc na zakupy
        lista_zakupow = self.initial_solution[i][1]
        stan_plecaka = 0
        produkty_do_wziecia = copy.deepcopy(self.lista_produktow)
        for j in range(len(lista_zakupow)):
          stan_plecaka += lista_zakupow[j]*self.lista_produktow[j][0]
          if lista_zakupow[j] == 1: # Bierzemy dany produkt jeśli 1
            produkty_do_wziecia[j][0] = self.max_poj_plecaka+1 # Zabronione przejscie

        # tym kroku spradzamy ponownie czy po wybraniu w pierwszej turze
        # x produktów i ich sumaryczna kaloryczność < zapotrzebowanie dzienne
        # to wtedy bierzemy n produktów o najmniejszej możliwej dopuszczalnej
        # masie tak aby sumaryczna_kalorycznosć >= zapotrzebowanie_dzienne
        niedobor = self.initial_solution[i][2]
        while niedobor < 0:
          min_waga = np.min(produkty_do_wziecia[:, 0])
          itemindex = np.where(produkty_do_wziecia[:, 0]==min_waga)[0][0]
          self.initial_solution[i][1][itemindex] = 1
          niedobor += produkty_do_wziecia[itemindex][1]
          produkty_do_wziecia[itemindex][0] = self.max_poj_plecaka+1
        self.initial_solution[i][2] = 0
      else:
        continue


  # def step1(self, macierz_pom_produktow2 = None):
  #   """
  #   Generowanie sąsiednich rozwiązan
  #   """ 
  #   macierz_pom_produktow = copy.deepcopy(macierz_pom_produktow2)

  #   if macierz_pom_produktow is None:
  #     macierz_pom_produktow = np.empty((len(self.initial_solution), 10))
  #     for i in range(0, len(self.initial_solution)):
  #       macierz_pom_produktow[i] = self.initial_solution[i][1] 

  #   for i in range(10):
  #     x_idx = np.random.randint(0, macierz_pom_produktow.shape[1])
  #     y_idx = np.random.randint(0, macierz_pom_produktow.shape[0])
  #     if macierz_pom_produktow[y_idx,x_idx] == 1:
  #       macierz_pom_produktow[y_idx,x_idx] = 0
  #   return macierz_pom_produktow


  def step1(self, macierz_pom_produktow = None):
    """
    Generowanie sąsiednich rozwiązan
    """ 
    if macierz_pom_produktow is None:
      macierz_pom_produktow = np.empty((len(self.initial_solution), 10))
      for i in range(0, len(self.initial_solution)):
        macierz_pom_produktow[i] = self.initial_solution[i][1] 

    for i in range(50):
      kierunek_przesuniecia = [1, 2, 3, 4] # 1 - gora, 2 - dol, 3-lewo, 4- 
      x_idx = np.random.randint(0, macierz_pom_produktow.shape[1])
      y_idx = np.random.randint(0, macierz_pom_produktow.shape[0])
      # print(x_idx, y_idx)
      if x_idx == 0: 
        kierunek_przesuniecia.remove(4)
      elif x_idx == macierz_pom_produktow.shape[1]-1:
        kierunek_przesuniecia.remove(2)

      if y_idx == 0:
        kierunek_przesuniecia.remove(1)

      elif y_idx == macierz_pom_produktow.shape[0]-1:
        kierunek_przesuniecia.remove(3)

      kierunek_przesuniecia_wybor = random.choice(kierunek_przesuniecia)
      # print(kierunek_przesuniecia_wybor)
      if kierunek_przesuniecia_wybor == 1: #gora
        macierz_pom_produktow[y_idx,x_idx], macierz_pom_produktow[y_idx - 1,x_idx] = macierz_pom_produktow[y_idx - 1 ,x_idx], macierz_pom_produktow[y_idx ,x_idx]
      elif kierunek_przesuniecia_wybor == 3:#dol
        macierz_pom_produktow[y_idx,x_idx], macierz_pom_produktow[y_idx + 1,x_idx] = macierz_pom_produktow[y_idx + 1 ,x_idx], macierz_pom_produktow[y_idx ,x_idx]

      elif kierunek_przesuniecia_wybor == 2: # prawo
        macierz_pom_produktow[y_idx,x_idx], macierz_pom_produktow[y_idx ,x_idx + 1] = macierz_pom_produktow[y_idx ,x_idx + 1], macierz_pom_produktow[y_idx ,x_idx]
      else: #lewo
        macierz_pom_produktow[y_idx,x_idx], macierz_pom_produktow[y_idx ,x_idx - 1] = macierz_pom_produktow[y_idx ,x_idx - 1], macierz_pom_produktow[y_idx ,x_idx]

    # self.tabu_list.append(copy.deepcopy(macierz_pom_produktow))
    return macierz_pom_produktow


  def check_weight(self, macierz_pom_produktow: np.ndarray) -> List[bool]:
    """
    jako parametr przyjmuje macierz w której wiersze reprezentują kolejne dni, natomiast
    kolumny listę produktów
    Zwraca List[bool] informujaca czy w danym dniu zakres plecaka został przekroczony
    """
    ponad_stan_lst = []
    for row in range(len(macierz_pom_produktow)):
      waga = 0
      for col in range(len(macierz_pom_produktow[row])):
        waga += macierz_pom_produktow[row][col] * self.lista_produktow[col][0]

      if waga > self.max_poj_plecaka:
        ponad_stan_lst.append(True)
      
      else:
        ponad_stan_lst.append(False)
        
    return ponad_stan_lst

  def check_capacity(self, macierz_pom_produktow2 = None):
    """
    jako parametr przyjmuje macierz w której wiersze reprezentują kolejne dni, natomiast
    kolumny listę produktów
    Zwraca List informujaca czy w danym dniu zakres lodowki został przekroczony
    """
    if macierz_pom_produktow2 is None:
      macierz_pom_produktow2 = np.empty((len(self.initial_solution), 10))
      for i in range(0, len(self.initial_solution)):
        macierz_pom_produktow2[i] = self.initial_solution[i][1]  
      
    ponad_stan_lst_lodowka = []
    bilans_kalorie = []
    aktualny_stan_lodowki = self.poczatkowy_stan_lodowki
    zawartosc_lodowki = []
    for row in range(len(macierz_pom_produktow2)):
      
      # print("nowy dzien, lista zakupow")
      # print(macierz_pom_produktow2[row])      
      if not all([v == 0 for v in macierz_pom_produktow2[row]]):
          zawartosc_lodowki.append(macierz_pom_produktow2[row])
          aktualny_stan_lodowki += sum(macierz_pom_produktow2[row])
          
          # print("dodaje ", sum(macierz_pom_produktow2[row]), " produktow")
          
          ponad_stan_lst_lodowka.append(aktualny_stan_lodowki)

      aktualne_zuzycie = 0
      bilans_kalorie.append(0)
      while aktualne_zuzycie < self.zapotrz_kal:
        if (len(np.nonzero(zawartosc_lodowki)[0])) > 0:
          # print(zawartosc_lodowki)
          # print(np.nonzero(zawartosc_lodowki))
          id_row = np.nonzero(zawartosc_lodowki)[0][0]
          id_col = np.nonzero(zawartosc_lodowki)[1][0]
          jedzony_produkt = self.lista_produktow[id_col]

          zawartosc_lodowki[id_row][id_col] = 0
          # if all([v == 0 for v in zawartosc_lodowki[0]]):
          #   if len(zawartosc_lodowki)==1:
          #     zawartosc_lodowki = np.zeros((1, 10))
          #   else:
          #     zawartosc_lodowki = np.delete(zawartosc_lodowki, 0, 0)
          kalorycznosc = jedzony_produkt[1]
          aktualne_zuzycie += kalorycznosc
          aktualny_stan_lodowki -= 1
      
          # print("usuwam 1 produkt")
      
        else:
          bilans_kalorie.append(aktualne_zuzycie - self.zapotrz_kal)
          break

    return ponad_stan_lst_lodowka, bilans_kalorie


  def check_current_sol_in_tabu_list(self, list_of_sol: List[np.ndarray], current_solution: np.ndarray) -> bool:
    """
    Sprawdza czy obecne rozwiązanie jest w tabu list zwraca True jeśli tak
    """
    # if len(self.tabu_list) == 1:
    #   return current_solution is self.tabu_list[0]
    
    # else:
    #   for elem in self.tabu_list:
    #     if current_solution is elem:
    #       return True
    #     else:
    #       continue
    #   return False
    return next((True for elem in self.tabu_list if elem is current_solution), False)

  # def check_current_sol_in_tabu_list(self, list_of_sol, current_solution: np.ndarray) -> bool:

  #   list_of_solution: List[bool] = []
  #   for elem in range(len(list_of_sol)):
  #       for i in range(len(list_of_sol[elem])):
  #           same_elem: bool = True
  #           for j in range(len(list_of_sol[elem][i])):
  #               if current_solution[i][j] != list_of_sol[elem][i][j]:
  #                   same_elem = False
  #                   break
  #               else:
  #                   same_elem = True
  #           if same_elem is False:
  #               break
  #       if same_elem is False:
  #           list_of_solution.append(False)
  #       else:
  #           list_of_solution.append(True)
  #   return any(list_of_solution)
        

  def zwroc_najlepsze_rozwiazanie(self, initial_solution):
    min = 0
    for i in range(len(initial_solution)):
      min += initial_solution[i][0]   
    return min

  def zwroc_liste_produktow(self, macierz_pom_produktow = None):

    if macierz_pom_produktow is not None:
      macierz_pom_produktow2 = np.empty((len(macierz_pom_produktow), 10))
      for i in range(0, len(macierz_pom_produktow)):
        macierz_pom_produktow2[i] = macierz_pom_produktow[i][1] 
      return macierz_pom_produktow2 
    
    else:
      macierz_pom_produktow = np.empty((len(self.initial_solution), 10))
      for i in range(0, len(self.initial_solution)):
        macierz_pom_produktow[i] = self.initial_solution[i][1] 
      return macierz_pom_produktow 

  def postac_do_rozwiazania(self, lista_produktow):
    lista = copy.deepcopy(lista_produktow)
    rozwiazanie = []
    bilans = self.check_capacity(lista)[1]
    for i in range(len(lista_produktow)):
      decyzja = 1
      if all([v == 0 for v in lista_produktow[i]]):
        decyzja = 0
      rozwiazanie.append([decyzja, lista_produktow[i], bilans[i]])
    return rozwiazanie


  def tabu_solution(self):
    it =  0
    lista_produktow_poczatkowa = self.zwroc_liste_produktow()
    self.tabu_list.append(lista_produktow_poczatkowa)
    print("Poczatkowe najlepsze rozwiazanie", self.zwroc_najlepsze_rozwiazanie(self.postac_do_rozwiazania(lista_produktow_poczatkowa)))
    print(self.postac_do_rozwiazania(lista_produktow_poczatkowa))
    # print("przed while", it)
    poprzednie_rozwiazanie = self.zwroc_liste_produktow()

    while(it < self.kryterium_stopu):
      it += 1
      # print("Po while", it)
      copy_for_step = copy.deepcopy(poprzednie_rozwiazanie)
      sasiednie_rozwiazanie = self.step1(copy_for_step)
      # self.check_kalorie()
      sasiednie_rozwiazanie = self.zwroc_liste_produktow(sasiednie_rozwiazanie)
      for_check = copy.deepcopy(sasiednie_rozwiazanie)
      sasiednie_rozwiazanie_lodowka_ponad_stan, kalorie_sasiednie_rozwiazanie = self.check_capacity(macierz_pom_produktow2=for_check)

      if self.check_current_sol_in_tabu_list(self.tabu_list, for_check):
        print("rozw w liscie tabu \t", it)
        continue

      elif any([v > self.maksymalna_poj_lodowki for v in sasiednie_rozwiazanie_lodowka_ponad_stan]) or any([v < 0 for v in kalorie_sasiednie_rozwiazanie]):
          if any([v < 0 for v in kalorie_sasiednie_rozwiazanie]) :
            print("wykrylo ze jest nie ok kcal")
          else:
            print("wykrylo ze jest nie ok else")
          self.tabu_list.append(for_check)
          continue

      else:
        print("rowz NIE w liscie tabu \t", it)
        rozwiazanie = self.postac_do_rozwiazania(sasiednie_rozwiazanie)
        best_current_sol = self.zwroc_najlepsze_rozwiazanie(rozwiazanie)
        poprzednie_rozwiazanie = self.zwroc_liste_produktow(rozwiazanie)

        if self.zwroc_najlepsze_rozwiazanie(self.best_solution) > best_current_sol:
          print("Najlepsze rozwiazanie\t: ", best_current_sol)
          print(rozwiazanie)
          self.best_solution = rozwiazanie
          print("Kolejne rozwiazania: \t", best_current_sol)
          print(rozwiazanie)
        else:
          print("NIE najlepsze rozwiazanie\t", best_current_sol)
          print(rozwiazanie)
          continue
      

# Generowanie kalendarza

Posiadając listę dniu w których sklepy są zamknięte jako lista_swiat, generujemy kalendarz przekazujac funkcji returncalendar następujące parametry: </br>
- first_day
- first_month
- first_year
- last_day
- last_month
- last_year

I zwraca kalendarz w postaci krotki: </br>
- data: obiekt datetime
- dzien tygodnia: int
- czy jest możliwość zakupów max_poj_plecaka-tak, 0-nie

In [8]:
# funkcja
import calendar
import datetime

# lista_swiat = [datetime.date(2022, 1, 1),
#                datetime.date(2022, 1, 6),
#                datetime.date(2022, 4, 17),
#                datetime.date(2022, 4, 18),
#                datetime.date(2022, 5, 1),
#                datetime.date(2022, 5, 3),
#                datetime.date(2022, 6, 5),
#                datetime.date(2022, 6, 16),
#                datetime.date(2022, 7, 15),
#                datetime.date(2022, 11, 1),
#                datetime.date(2022, 11, 11),
#                datetime.date(2022, 12, 25),
#                datetime.date(2022, 12, 26)]
lista_swiat = []


def returncalendar(first_day, first_month, first_year, last_day, last_month, last_year) :
  #doesnt include the last day -> update po dodaniu days = 1 tak
  #lista dni iteruje od 0

  # wielkanoc_mth = input("Kiedy Wielkanoc: numer miesiąca")
  # wielkanoc_day = input("Kiedy Wielkanoc: numer dnia")
  # boze_cialo_mth = input("Kiedy Boże Ciało: numer miesiąca")
  # boze_cialo_day = input("Kiedy Boże Ciało: numer dnia")

 # roboczo zeby nie wpisywac
  wielkanoc_mth = 4
  wielkanoc_day = 17
  boze_cialo_mth = 6
  boze_cialo_day = 16


  # lista_swiat_here = lista_swiat.copy()
  # lista_swiat_here.append((int(wielkanoc_mth), int(wielkanoc_day)))
  # lista_swiat_here.append((int(boze_cialo_mth), int(boze_cialo_day)))
  # # print(lista_swiat_here)

  # lista_swiat_here.sort()
  # # print(lista_swiat_here)


  vector_days = []

  d0 = datetime.date(first_year, first_month, first_day)
  d1 = datetime.date(last_year, last_month, last_day)

  start_date = d0
  end_date = d1
  delta = datetime.timedelta(days=1)

  while start_date <= end_date:
    weight = lodowka.max_poj_plecaka
    # if start_date.weekday() == 6:
    #   weight = 0

    if start_date in lista_swiat:
      weight = 0  # poj plecaka

    vector_days.append((start_date, start_date.weekday(), weight))

    start_date += delta

  return vector_days


terminarz = returncalendar(3, 1, 2022, 15, 1, 2022)
for i in range(len(terminarz)):
  print(terminarz[i], '\n')



(datetime.date(2022, 1, 3), 0, 7) 

(datetime.date(2022, 1, 4), 1, 7) 

(datetime.date(2022, 1, 5), 2, 7) 

(datetime.date(2022, 1, 6), 3, 7) 

(datetime.date(2022, 1, 7), 4, 7) 

(datetime.date(2022, 1, 8), 5, 7) 

(datetime.date(2022, 1, 9), 6, 7) 

(datetime.date(2022, 1, 10), 0, 7) 

(datetime.date(2022, 1, 11), 1, 7) 

(datetime.date(2022, 1, 12), 2, 7) 

(datetime.date(2022, 1, 13), 3, 7) 

(datetime.date(2022, 1, 14), 4, 7) 

(datetime.date(2022, 1, 15), 5, 7) 



In [9]:
l1 = lodowka(terminarz, df)

l1.tabu_solution()
# tab = []
# for i in range(len(l1.initial_solution)):
#   tab.append(l1.initial_solution[i][1])

# print(tab)
# print(l1.postac_do_rozwiazania(tab))
# l1.check_kalorie()
# print('\n')
# for i in range(len(l1.initial_solution)):
#   print(l1.initial_solution[i])

# to_print = l1.step1()
# print(to_print)

Poczatkowe najlepsze rozwiazanie 13
[[1, array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 0], [1, array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]), 0], [1, array([1., 1., 1., 0., 0., 0., 1., 0., 1., 1.]), 0], [1, array([1., 0., 0., 0., 0., 1., 0., 0., 0., 1.]), 0], [1, array([1., 0., 0., 0., 0., 0., 0., 1., 0., 1.]), 0], [1, array([0., 1., 0., 1., 0., 0., 0., 0., 1., 0.]), 0], [1, array([0., 0., 1., 0., 1., 0., 0., 0., 0., 1.]), 0], [1, array([1., 0., 0., 0., 0., 0., 0., 1., 0., 1.]), 0], [1, array([0., 1., 0., 1., 0., 0., 1., 0., 1., 0.]), 0], [1, array([0., 1., 0., 0., 1., 1., 0., 0., 0., 0.]), 0], [1, array([1., 0., 0., 0., 1., 0., 0., 1., 0., 0.]), 0], [1, array([1., 0., 0., 0., 1., 0., 0., 1., 1., 0.]), 0], [1, array([1., 1., 0., 0., 0., 0., 0., 0., 1., 0.]), 0]]
wykrylo ze jest nie ok else
wykrylo ze jest nie ok else
wykrylo ze jest nie ok else
wykrylo ze jest nie ok else
wykrylo ze jest nie ok else
wykrylo ze jest nie ok else
wykrylo ze jest nie ok else
wykrylo ze jest nie ok els

In [10]:
# print("Rozw1\t", to_print)
# l1.tabu_list.append(to_print)
# print("tabu list1:\t", l1.tabu_list)
# print(l1.check_current_sol_in_tabu_list(l1.tabu_list, to_print))
# to_print_1 = l1.step1()
# print("Rozw2\t", to_print_1)
# print("tabu list1:\t", l1.tabu_list)
# print(l1.check_current_sol_in_tabu_list(l1.tabu_list, to_print_1))


In [11]:
# print(to_print.all() == to_print_1.all())

In [12]:
# Test check solution in tabu list
lst = []
lst_1 = np.array([[0, 1, 1, 0, 1], [0, 1, 1, 0 , 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]])
lst_2 = np.array([[0, 1, 1, 0, 1], [0, 1, 1, 1 , 0], [0, 1, 0, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]])
lst.append(lst_1)
lst.append(lst_2)
l2 = lodowka(terminarz, df)
l2.tabu_list = lst
print(lst_1)
print(lst)
print(l2.check_current_sol_in_tabu_list(l2.tabu_list, lst_1))
# lst_2 = np.array([[0, 0, 1, 0, 1], [0, 1, 1, 0 , 0], [0, 1, 1, 1, 0], [0, 0, 0, 1, 0], [1, 1, 1, 0, 1]])
print(lst_2)

print(l2.check_current_sol_in_tabu_list(l2.tabu_list, lst_2))
print(lst)

[[0 1 1 0 1]
 [0 1 1 0 0]
 [0 1 1 1 0]
 [0 0 0 1 0]
 [1 1 1 0 1]]
[array([[0, 1, 1, 0, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 0, 1]]), array([[0, 1, 1, 0, 1],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 0, 1]])]
True
[[0 1 1 0 1]
 [0 1 1 1 0]
 [0 1 0 1 0]
 [0 0 0 1 0]
 [1 1 1 0 1]]
True
[array([[0, 1, 1, 0, 1],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 0, 1]]), array([[0, 1, 1, 0, 1],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 0, 1]])]


In [13]:
# l1.check_weight(to_print)

In [14]:
l1.check_capacity()

([10.0,
  17.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Garbage



In [15]:
## easter
from dateutil.easter import *
from datetime import timedelta
year = 1994
print((easter(year)))
print((easter(year)+timedelta(60)))


1994-04-03
1994-06-02


In [16]:
# Tabu Search

# STEP 0

# def generate_initial_solution(terminarz, lista_produktow) -> np.ndarray:
#   initial_solution = []
#   lista_produktow = lista_produktow.to_numpy()
#   for i in range(len(terminarz)):
#     if terminarz[i][2] != 0:
#       initial_solution.append([ np.random.randint(0, 2), [] ])
#     else:
#       initial_solution.append([ 0, [] ]) 

#   for i in range(len(initial_solution)):
#     if initial_solution[i][0] != 0:
#       for j in range(len(lista_produktow)):
#         initial_solution[i][1].append(np.random.randint(0, 2)) #ewentualnie mozna brac wiecej (randint (0, x))
  
  
  
#   return initial_solution

# generate_initial_solution(terminarz, df)


In [17]:

#### GARBAGE
  # obj = calendar.Calendar()
  # obj1 = calendar.TextCalendar()
  # print(obj1.formatmonth(year, month))

  # for day in obj.itermonthdays(year, month):
  #   print(day)

# year, month = 2017, 3
# obj = calendar.Calendar()
# obj1 = calendar.TextCalendar()
# print(obj1.formatmonth(year, month))

# for day in obj.itermonthdays(year, month):
#   print(day)
# deck = list(range(0, 10))
# deck = list(range(1, 10001))
# random.shuffle(deck)
import random
random.sample(range(10), 10)

[6, 0, 5, 3, 8, 4, 2, 7, 1, 9]

In [18]:
# np.random.randint(1, 5) 
len(l1.initial_solution)

13

In [23]:
# Step 1. Create an initial population of P chromosomes.
def init_population(n, terminarz, lista_produktow):
  """
  Parameters:
  n - ilość osobników do otrzymania
  terminarz - lista dni na jakie chcemy zrobić rozpiskę
  lista_produktow - stała lista produktów z ich wagą i kalorycznością

  Returns:
  init_population (list): lista rozwiązań początkowych
  """
  init_population = []
  for i in range(n):
    lodowka_ = lodowka(terminarz, lista_produktow).initial_solution
    init_population.append(lodowka_)
  return init_population

# Step 2. Evaluate the fitness of each chromosome.
def evaluate_chromosome(solution):
  min = 0
  for i in range(len(solution)):
    min += solution[i][0]   
  return min

def eval_init_population(init_population):
  evaluated_init_population = []
  for i in range(len(init_population)):
    evaluated_init_population.append((init_population[i], evaluate_chromosome(init_population[i])))
  return evaluated_init_population


# Step 3. Choose P/2 parents from the current population via proportional selection.
def rulette(evaluated_init_population):
  parents_list = []
  iter = 0
  list_eval = copy.deepcopy(evaluated_init_population)

  how_many = len(evaluated_init_population) // 2
  if how_many%2!= 0:
    how_many += 1

  while iter < how_many:
    iter += 1
    suma_po_wyjsciach = 0
    for i in range(len(list_eval)):
      suma_po_wyjsciach += list_eval[i][1]
      
    stop_points = [0]
    for j in range(len(list_eval)):
      stop_points.append(1/list_eval[j][1])
    sum_stop_points = sum(stop_points)
    normalized_stop_points = [point/sum_stop_points for point in stop_points]
    normalized_stop_points_summed = [normalized_stop_points[0]]
    for i in range(1, len(normalized_stop_points)):
      normalized_stop_points_summed.append(normalized_stop_points_summed[i-1]+normalized_stop_points[i])

    rand = random.random()
    chosen_point = 0 #indeks rodzica
    for i in range(len(normalized_stop_points_summed)-1):
      if normalized_stop_points_summed[i] <= rand < normalized_stop_points_summed[i+1]:        
        chosen_point = i
        break
      else:
        continue
    parents_list.append(list_eval.pop(chosen_point))

  return parents_list
  

# Step 4. Randomly select two parents to create offspring using crossover operator.
def return_pairs():
  pass
# Step 5. Apply mutation operators for minor changes in the results.

# Step 6. Repeat Steps  4 and 5 until all parents are selected and mated.

# Step 7. Replace old population of chromosomes with new one.

# Step 8. Evaluate the fitness of each chromosome in the new population.

# Step 9. Terminate if the number of generations meets some upper bound; otherwise go to Step  3.

lista_produktow = np.array([
      [5.600e-01, 1.335e+03],
      [2.500e-01, 1.274e+03],
      [3.800e-01, 6.570e+02],
      [7.000e-02, 1.258e+03],
      [1.100e-01, 8.410e+02],
      [5.500e-01, 1.146e+03],
      [3.200e-01, 9.850e+02],
      [2.000e-02, 8.490e+02],
      [1.400e-01, 1.235e+03],
      [1.000e-01, 1.343e+03]])

terminarz = returncalendar(3, 1, 2022, 15, 1, 2022)

sol = init_population(10,terminarz, lista_produktow)

# for i in range(len(sol)):
#   print(sol[i])

sol_2= eval_init_population(sol)

# for i in range(len(sol)):
#   print(sol_2[i])

rulette(sol_2)

10
9
8
7
6
5


[([[1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 0],
   [1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 0],
   [1, [1, 0, 1, 0, 0, 1, 1, 0, 1, 1], 0],
   [1, [0, 0, 0, 0, 0, 1, 1, 0, 1, 0], 0],
   [1, [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], 0],
   [1, [0, 0, 1, 1, 0, 0, 1, 1, 0, 0], 0],
   [1, [1, 0, 0, 0, 0, 0, 0, 1, 1, 0], 0],
   [1, [0, 0, 0, 1, 1, 0, 0, 0, 0, 1], 0],
   [1, [0, 0, 0, 1, 0, 1, 0, 0, 0, 1], 0],
   [1, [0, 0, 1, 1, 0, 1, 0, 0, 0, 0], 0],
   [1, [0, 1, 0, 0, 1, 0, 1, 0, 0, 0], 0],
   [1, [0, 0, 0, 0, 1, 0, 0, 1, 0, 1], 0],
   [1, [0, 0, 1, 0, 0, 1, 0, 1, 0, 0], 0]],
  13),
 ([[1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 0],
   [1, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 0],
   [1, [0, 1, 0, 1, 1, 0, 1, 0, 1, 1], 0],
   [1, [1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 0],
   [1, [0, 0, 0, 0, 0, 1, 0, 1, 1, 0], 0],
   [1, [1, 0, 1, 0, 0, 0, 1, 0, 0, 1], 0],
   [1, [0, 0, 0, 0, 1, 0, 1, 0, 1, 0], 0],
   [1, [1, 0, 0, 0, 0, 0, 0, 0, 1, 1], 0],
   [1, [0, 0, 0, 0, 1, 0, 0, 0, 1, 1], 0],
   [1, [1, 1, 0, 0, 0, 0, 0, 0, 0, 1], 0],
   